In [239]:
gpu=5
CUDA_VISIBLE_DEVICES=gpu


In [240]:
import numpy as np
import pandas as pd

import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data.sampler import SubsetRandomSampler

import matplotlib.pyplot as plt
import time
import copy
from random import shuffle

import tqdm.notebook as tqdm

import sklearn
from sklearn.metrics import accuracy_score, cohen_kappa_score
from sklearn.metrics import classification_report
from PIL import Image
import cv2

# import osa
import shutil

In [241]:
import numpy as np
import pandas as pd

import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data.sampler import SubsetRandomSampler

import matplotlib.pyplot as plt
import time
import copy
from random import shuffle

import tqdm.notebook as tqdm

import sklearn
from sklearn.metrics import accuracy_score, cohen_kappa_score
from sklearn.metrics import classification_report
from PIL import Image
import cv2

# import osa
import shutil

In [242]:
df =pd.read_csv('/DATA/chowdari1/DATA/dataset/nih/Data_Entry_2017.csv')

In [243]:
df.head()

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Unnamed: 11
0,00000001_000.png,Cardiomegaly,0,1,58,M,PA,2682,2749,0.143,0.143,NaN
1,00000001_001.png,Cardiomegaly|Emphysema,1,1,58,M,PA,2894,2729,0.143,0.143,NaN
2,00000001_002.png,Cardiomegaly|Effusion,2,1,58,M,PA,2500,2048,0.168,0.168,NaN
3,00000002_000.png,No Finding,0,2,81,M,PA,2500,2048,0.171,0.171,NaN
4,00000003_000.png,Hernia,0,3,81,F,PA,2582,2991,0.143,0.143,NaN


In [244]:
import glob
my_glob = glob.glob('/DATA/chowdari1/DATA/dataset/nih/images*/images/*.png')
print('Number of Observations: ', len(my_glob))

Number of Observations:  112120


In [245]:
my_glob.sort()

In [246]:
df["path"]=my_glob

In [247]:
df.head()

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Unnamed: 11,path
0,00000001_000.png,Cardiomegaly,0,1,58,M,PA,2682,2749,0.143,0.143,NaN,/DATA/chowdari1/DATA/dataset/nih/images_001/im...
1,00000001_001.png,Cardiomegaly|Emphysema,1,1,58,M,PA,2894,2729,0.143,0.143,NaN,/DATA/chowdari1/DATA/dataset/nih/images_001/im...
2,00000001_002.png,Cardiomegaly|Effusion,2,1,58,M,PA,2500,2048,0.168,0.168,NaN,/DATA/chowdari1/DATA/dataset/nih/images_001/im...
3,00000002_000.png,No Finding,0,2,81,M,PA,2500,2048,0.171,0.171,NaN,/DATA/chowdari1/DATA/dataset/nih/images_001/im...
4,00000003_000.png,Hernia,0,3,81,F,PA,2582,2991,0.143,0.143,NaN,/DATA/chowdari1/DATA/dataset/nih/images_001/im...


In [248]:
df['Image Index'][3], df['path'][3]

('00000002_000.png',
 '/DATA/chowdari1/DATA/dataset/nih/images_001/images/00000002_000.png')

In [249]:
target=[]
for i in range(df.shape[0]):
    if(df["Finding Labels"][i]=="No Finding"):
        target.append(0)
    else:
        target.append(1)

target=np.array(target)
# test=pd.read_csv('/DATA/chowdari1/DATA/dataset/stanford/CheXpert-v1.0-small/valid.csv')

In [250]:
df['target']=target

In [251]:

with open("/DATA/chowdari1/DATA/dataset/nih/paper_dataset_split/train.txt", "r") as file:
	train_lines = file.readlines()
	# print(lines)
len(train_lines)

8574

In [252]:

with open("/DATA/chowdari1/DATA/dataset/nih/paper_dataset_split/val.txt", "r") as file:
	val_lines = file.readlines()
	# print(lines)
len(val_lines)



1706

In [253]:
val_lines.sort()

In [254]:

with open("/DATA/chowdari1/DATA/dataset/nih/paper_dataset_split/test_attending_rad.txt", "r") as file:
	test1_lines = file.readlines()
	print(len(test1_lines))
len(train_lines)
with open("/DATA/chowdari1/DATA/dataset/nih/paper_dataset_split/test_rad_consensus_voted3.txt", "r") as file:
	test2_lines = file.readlines()
	print(len(test2_lines))
# len(train_lines)+
if(test1_lines!=test2_lines):
    print('wrong')
wrong=0
for i in range(1344):
	if(test1_lines[i]!=test2_lines[i]):
		wrong+=1
		# print(i,test1_lines[i],test2_lines[i])
wrong

1344
1344
wrong


23

In [255]:
test1_lines.sort()
test2_lines.sort()

In [256]:
train_lines.sort()

In [258]:
train_lines[1][:-3]

'00000045_000.png'

In [259]:
df['Image Index'][1]

'00000001_001.png'

In [260]:
x = np.empty(112120, dtype=object)
df["id"]=x
df["id"]=df["id"].fillna("d")
df.shape

(112120, 15)

In [261]:
def substr(s1,s2):
    for i in range(len(s1)):
        if(s1[i]!=s2[i]):
            return False
    return True

In [262]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

In [263]:
df['id'].shape

(112120,)

In [265]:
count=0
wrong=0
for i in range(0,112120):
    if(i==112119):
        print(count)

    if(substr(train_lines[count][:-3],df["Image Index"][i])):
        df["id"][i]="train"
        if(df['target'][i]!= int(train_lines[count][-3:-1])):
            wrong+=1
        count+=1
    if(count==8574):
        break
print('count: ',count)
wrong

count:  8574


0

In [269]:
testcount=0
testwrong=0
valcount=0
valwrong=0
for i in range(0,112120):
    if(i==112119):
        print(count,testcount,valcount)

    if(valcount<1706 and substr(val_lines[valcount][:-3],df["Image Index"][i])):
        df["id"][i]="val"
        if(df['target'][i]!= int(val_lines[valcount][-3:-1])):
            df['target'][i]= int(val_lines[valcount][-3:-1])
            valwrong+=1
            
            a=i
        valcount+=1
    if(testcount<1344 and substr(test1_lines[testcount][:-3],df["Image Index"][i])):
        df["id"][i]="test"
        if(test1_lines[testcount][-2:-1]==' '):
            test1_lines[testcount]+='\n'
        if(df['target'][i]!= int(test1_lines[testcount][-2:-1])):
            df['target'][i]= int(test1_lines[testcount][-2:-1])
            testwrong+=1
            # print(test1_lines[testcount],df['Image Index'][i],df['target'][i],i)
            b=i
        testcount+=1
print('test: ',testcount, testwrong)
valcount,valwrong,a,b

8574 1344 1706
test:  1344 21


(1706, 9, 107756, 108454)

In [271]:

train_df=pd.DataFrame()
train_df["Image index"]= np.empty(8574,dtype=object)
train_df["target"]= np.empty(8574,dtype=int)
train_df["path"]= np.empty(8574,dtype=object)

val_df=pd.DataFrame()
val_df["Image index"]= np.empty(1706,dtype=object)
val_df["target"]= np.empty(1706,dtype=int)
val_df["path"]= np.empty(1706,dtype=object)

test_df=pd.DataFrame()
test_df["Image index"]= np.empty(1344,dtype=object)
test_df["target"]= np.empty(1344,dtype=int)
test_df["path"]= np.empty(1344,dtype=object)


In [272]:
count_train=0
count_test=0
count_val=0
for i in range(0,len(df["id"])):
    if(count_train<8574 and df["id"][i]=="train" ):
        train_df["target"][count_train]=df["target"][i]
        train_df["path"][count_train]=df["path"][i]
        train_df["Image index"][count_train]=df["Image Index"][i]
        count_train+=1
    elif(count_val<1706 and df["id"][i]=="val" ):
        val_df["target"][count_val]=df["target"][i]
        val_df["path"][count_val]=df["path"][i]
        val_df["Image index"][count_val]=df["Image Index"][i]
        count_val+=1
    # elif(count_train<25595 ):
    elif(count_test<1344 and df["id"][i]=="test" ):
        test_df["target"][count_test]=df["target"][i]
        test_df["path"][count_test]=df["path"][i]
        test_df["Image index"][count_test]=df["Image Index"][i]
        count_test+=1
count_train,count_test,count_val


(8574, 1344, 1706)

In [273]:
train_df.head()
display(train_df)

,Image index,target,path
0,00000038_003.png,1,/DATA/chowdari1/DATA/dataset/nih/images_001/im...
1,00000045_000.png,1,/DATA/chowdari1/DATA/dataset/nih/images_001/im...
2,00000047_001.png,0,/DATA/chowdari1/DATA/dataset/nih/images_001/im...
3,00000049_002.png,0,/DATA/chowdari1/DATA/dataset/nih/images_001/im...
4,00000054_001.png,0,/DATA/chowdari1/DATA/dataset/nih/images_001/im...
...,...,...,...
8569,00030740_000.png,0,/DATA/chowdari1/DATA/dataset/nih/images_012/im...
8570,00030786_001.png,1,/DATA/chowdari1/DATA/dataset/nih/images_012/im...
8571,00030786_002.png,1,/DATA/chowdari1/DATA/dataset/nih/images_012/im...
8572,00030786_003.png,1,/DATA/chowdari1/DATA/dataset/nih/images_012/im...


In [274]:
test_df.head()
display(test_df)

,Image index,target,path
0,00000013_025.png,1,/DATA/chowdari1/DATA/dataset/nih/images_001/im...
1,00000013_040.png,1,/DATA/chowdari1/DATA/dataset/nih/images_001/im...
2,00000032_029.png,1,/DATA/chowdari1/DATA/dataset/nih/images_001/im...
3,00000116_000.png,1,/DATA/chowdari1/DATA/dataset/nih/images_001/im...
4,00000116_016.png,1,/DATA/chowdari1/DATA/dataset/nih/images_001/im...
...,...,...,...
1339,00030782_000.png,0,/DATA/chowdari1/DATA/dataset/nih/images_012/im...
1340,00030788_000.png,0,/DATA/chowdari1/DATA/dataset/nih/images_012/im...
1341,00030791_000.png,0,/DATA/chowdari1/DATA/dataset/nih/images_012/im...
1342,00030794_000.png,0,/DATA/chowdari1/DATA/dataset/nih/images_012/im...


In [275]:
val_df.head()
display(val_df)

,Image index,target,path
0,00000009_000.png,1,/DATA/chowdari1/DATA/dataset/nih/images_001/im...
1,00000071_004.png,1,/DATA/chowdari1/DATA/dataset/nih/images_001/im...
2,00000071_006.png,1,/DATA/chowdari1/DATA/dataset/nih/images_001/im...
3,00000099_000.png,1,/DATA/chowdari1/DATA/dataset/nih/images_001/im...
4,00000099_001.png,1,/DATA/chowdari1/DATA/dataset/nih/images_001/im...
...,...,...,...
1701,00030561_003.png,1,/DATA/chowdari1/DATA/dataset/nih/images_012/im...
1702,00030561_004.png,1,/DATA/chowdari1/DATA/dataset/nih/images_012/im...
1703,00030561_005.png,1,/DATA/chowdari1/DATA/dataset/nih/images_012/im...
1704,00030561_007.png,1,/DATA/chowdari1/DATA/dataset/nih/images_012/im...


In [197]:
path="/DATA/chowdari1/DATA/csv/"
test_df.to_csv(path+"ppr_nih_test.csv")

In [198]:
path="/DATA/chowdari1/DATA/csv/"
train_df.to_csv(path+"ppr_nih_train.csv")

In [199]:
path="/DATA/chowdari1/DATA/csv/"
val_df.to_csv(path+"ppr_nih_val.csv")